# KNN ENTRENAMIENTO

In [113]:
import pandas as pd
import numpy as np
import pandas as df
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

Al igual que cuando planteamos el EDA, preprocesamos los datos y hacemos las subdivisiones adecuadas en datos de entranamiento y test

In [114]:
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip", sep=",")

In [115]:
sotavento_columns = [col for col in wind_ava.columns if ".13" in col]
sotavento_columns.insert(0, "datetime")
sotavento_columns.append("energy")
wind_ava_sotavento = wind_ava[sotavento_columns]

In [116]:
wind_ava_sotavento = wind_ava_sotavento.rename(columns = {
    "energy": "Energy", 
    "datetime": "Datetime", 
    "p54.162.13": "Vertical integral of temperature", 
    "p55.162.13": "Vertical integral of water vapour", 
    "cape.13": "Convective available potential energy", 
    "p59.162.13": "Vertical integral of divergence of kinetic energy", 
    "lai_lv.13": "Leaf area index, low vegetation", 
    "lai_hv.13": "Leaf area index, high vegetation", 
    "u10n.13": "Neutral wind at 10 m u-component", 
    "v10n.13": "Neutral wind at 10 m v-component", 
    "sp.13": "Surface pressure",
    "stl1.13": "Soil temperature level 1",
    "u10.13": "10 metre U wind component",
    "v10.13": "10 metre V wind component",
    "t2m.13": "2 metre temperature", 
    "stl2.13": "Soil temperature level 2", 
    "stl3.13": "Soil temperature level 3", 
    "iews.13": "Instantaneous eastward turbulent surface stress",
    "inss.13": "Instantaneous northward turbulent surface", 
    "stl4.13": "Soil temperature level 4", 
    "fsr.13": "Forecast surface roughness", 
    "flsr.13": "Forecast logarithm of surface roughness for heat",
    "u100.13": "100 metre U wind component",
    "v100.13": "100 metre V wind component", })
wind_ava_sotavento.head()

,Datetime,Vertical integral of temperature,Vertical integral of water vapour,Convective available potential energy,Vertical integral of divergence of kinetic energy,"Leaf area index, low vegetation","Leaf area index, high vegetation",Neutral wind at 10 m u-component,Neutral wind at 10 m v-component,Surface pressure,...,Soil temperature level 2,Soil temperature level 3,Instantaneous eastward turbulent surface stress,Instantaneous northward turbulent surface,Soil temperature level 4,Forecast surface roughness,Forecast logarithm of surface roughness for heat,100 metre U wind component,100 metre V wind component,Energy
0,2005-01-02 18:00:00,2.510824e+06,9.186295,13.527577,1.386937e+06,2.344111,2.432983,-0.757587,-1.922799,99846.319914,...,281.042026,281.462478,-0.057958,-0.138650,284.684755,0.404731,-5.927092,-1.780562,-4.443617,402.71
1,2005-01-03 00:00:00,2.513173e+06,8.849569,6.896412,1.153526e+06,2.343719,2.432838,-1.412620,-1.403011,99917.733093,...,280.747406,281.486541,-0.103576,-0.083050,284.667948,0.404920,-5.913881,-3.743344,-3.129469,696.80
2,2005-01-03 06:00:00,2.509627e+06,7.924080,4.774439,1.098754e+06,2.343300,2.432704,-2.290185,-0.754580,99764.378681,...,280.114863,281.487095,-0.165721,-0.036241,284.651914,0.405704,-5.908272,-5.097203,-1.157748,1591.15
3,2005-01-03 12:00:00,2.510571e+06,6.922709,0.000000,1.076021e+06,2.342830,2.432514,-3.497855,1.271028,99672.670459,...,279.991138,281.472435,-0.275550,0.098192,284.636266,0.403967,-5.961995,-4.500835,1.502478,1338.62
4,2005-01-03 18:00:00,2.505664e+06,6.646282,0.000000,1.070830e+06,2.342437,2.432369,-0.971249,0.553060,99372.811211,...,280.576898,281.473265,-0.056553,0.041844,284.620232,0.403808,-5.987860,-3.392324,2.131114,562.50


In [117]:
wind_ava_sotavento["Datetime"] = pd.to_datetime(wind_ava_sotavento["Datetime"])
wind_ava_sotavento.dtypes

Datetime                                             datetime64[ns]
Vertical integral of temperature                            float64
Vertical integral of water vapour                           float64
Convective available potential energy                       float64
Vertical integral of divergence of kinetic energy           float64
Leaf area index, low vegetation                             float64
Leaf area index, high vegetation                            float64
Neutral wind at 10 m u-component                            float64
Neutral wind at 10 m v-component                            float64
Surface pressure                                            float64
Soil temperature level 1                                    float64
10 metre U wind component                                   float64
10 metre V wind component                                   float64
2 metre temperature                                         float64
Soil temperature level 2                        

In [118]:
duplicate_rows = wind_ava_sotavento[wind_ava_sotavento.duplicated()]
print("Número de filas duplicadas: ", duplicate_rows.shape)

Número de filas duplicadas:  (0, 24)


In [119]:
null_rows = wind_ava_sotavento[wind_ava_sotavento.isnull().any(axis=1)]
print("Número de filas nulas: ", null_rows.shape)

Número de filas nulas:  (0, 24)


In [120]:
import copy
# Dividir la columna 'Datetime' en dos nuevas columnas: 'fecha' y 'año'
wind_ava_sotavento['Month'] = wind_ava_sotavento['Datetime'].dt.month
wind_ava_sotavento['Year'] = wind_ava_sotavento['Datetime'].dt.year

# Eliminar la columna 'Datetime'
wind_ava_sotavento.drop(columns=['Datetime'], inplace=True)
# Ordenar el DataFrame wind_ava_sotavento por 'Datetime'
wind_ava_sotavento.sort_values(by=['Year', 'Month'], inplace=True)
# Crear una copia profunda del DataFrame wind_ava_sotavento
dataset_1 = copy.deepcopy(wind_ava_sotavento)
# Eliminar la columna 'Energy' de dataset_1
del dataset_1['Energy']
# Crear una copia profunda del DataFrame wind_ava_sotavento
dataset_3 = copy.deepcopy(wind_ava_sotavento)
dataset_2 = dataset_3["Energy"]
dataset_1.info()
dataset_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4748 entries, 0 to 4747
Data columns (total 24 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Vertical integral of temperature                   4748 non-null   float64
 1   Vertical integral of water vapour                  4748 non-null   float64
 2   Convective available potential energy              4748 non-null   float64
 3   Vertical integral of divergence of kinetic energy  4748 non-null   float64
 4   Leaf area index, low vegetation                    4748 non-null   float64
 5   Leaf area index, high vegetation                   4748 non-null   float64
 6   Neutral wind at 10 m u-component                   4748 non-null   float64
 7   Neutral wind at 10 m v-component                   4748 non-null   float64
 8   Surface pressure                                   4748 non-null   float64
 9   Soil tem

In [121]:
dataset_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4748 entries, 0 to 4747
Data columns (total 24 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Vertical integral of temperature                   4748 non-null   float64
 1   Vertical integral of water vapour                  4748 non-null   float64
 2   Convective available potential energy              4748 non-null   float64
 3   Vertical integral of divergence of kinetic energy  4748 non-null   float64
 4   Leaf area index, low vegetation                    4748 non-null   float64
 5   Leaf area index, high vegetation                   4748 non-null   float64
 6   Neutral wind at 10 m u-component                   4748 non-null   float64
 7   Neutral wind at 10 m v-component                   4748 non-null   float64
 8   Surface pressure                                   4748 non-null   float64
 9   Soil tem

# Evaluación con hiperparámetros por omisión

# NO ENTIENDO MUY SI AQUI YA HACEMOS LA EVALUACION CON EL ESCALADOR QUE HEMOS ELEGIDO O SI NO CON KNN

In [122]:
# Para esta evaluación, se usará el modelo KNN con hiperparámetros por omisión.
# Se evaluará el modelo con el RMSE y MAE, además de la media de aciertos.
# Se evaluará el tiempo de ejecución.
# Para ello, establecemos un número de vecinos predeterminado

x_train, x_test, y_train, y_test = train_test_split(dataset_1, dataset_2, test_size=0.33, random_state=42)

t_inicio = time.time() 
knn = KNeighborsRegressor()

knn.fit(x_train, y_train)
accuracy_train_og = knn.score(x_test, y_test) 
t_fin = time.time()
print('El modelo KNN con hiperparámetros por omisión ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_og))

# Miramos sus errores
prediction = knn.predict(x_test)
error_rmse_og = sqrt(mean_squared_error(y_test, prediction))
error_mae_og = mean_absolute_error(y_test, prediction)
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_og, error_mae_og))
t_og = t_fin - t_inicio
print('El tiempo de ejecucion ha sido de {}'.format(t_og))

El modelo KNN con hiperparámetros por omisión ha obtenido una media de resultado de 0.08
Como errores:
	RMSE	= 637.8017280433622
	MAE	= 498.68136694320356
El tiempo de ejecucion ha sido de 0.01699519157409668


In [123]:
x_train, x_test, y_train, y_test = train_test_split(dataset_1, dataset_2, test_size=0.33, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

# Crear y entrenar el modelo KNN con hiperparámetros por omisión
knn = KNeighborsRegressor()
t_inicio = time.time() 
knn.fit(X_train_scaled, y_train)
t_fin = time.time()

# Evaluar el modelo
accuracy_train_og = knn.score(X_test_scaled, y_test)
print('El modelo KNN con hiperparámetros por omisión ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_og))

# Calcular errores
prediction = knn.predict(X_test_scaled)
error_rmse_og = sqrt(mean_squared_error(y_test, prediction))
error_mae_og = mean_absolute_error(y_test, prediction)
print('Como errores:\n\tRMSE\t= {:.2f}\n\tMAE\t= {:.2f}'.format(error_rmse_og, error_mae_og))

# Calcular el tiempo de ejecución
t_og = t_fin - t_inicio
print('El tiempo de ejecucion ha sido de {:.2f} segundos'.format(t_og))

El modelo KNN con hiperparámetros por omisión ha obtenido una media de resultado de 0.62
Como errores:
	RMSE	= 411.92
	MAE	= 298.95
El tiempo de ejecucion ha sido de 0.00 segundos


# Ajuste de hiperparametros

# Escalado

In [124]:
# Escalamos con MinMax de un rango entre 0 y 1
scaler_minmax = MinMaxScaler(feature_range=(0, 1))
x_train_minmax = scaler_minmax.fit_transform(x_train)
x_test_minmax = scaler_minmax.transform(x_test)

# Entrenamos de la misma manera que sin ajuste de hiperparámetros
t_inicio = time.time() # Guardamos el tiempo inicial
knn = KNeighborsRegressor()
#np.random.seed(42)  # Para la toma de decisión entre varios puntos
knn.fit(x_train_minmax, y_train)   # Se entrena el modelo
accuracy_train_minmax = knn.score(x_test_minmax, y_test)  # Se obtiene el resultado de aciertos
t_fin = time.time() # Guardamos el tiempo final
print('El modelo KNN con escalado MinMax ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_minmax))
# Miramos sus errores
prediction = knn.predict(x_test_minmax)
error_rmse_minmax = sqrt(mean_squared_error(y_test, prediction))
error_mae_minmax = mean_absolute_error(y_test, prediction)
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_minmax, error_mae_minmax))
t_minmax = t_fin - t_inicio
print('El tiempo de ejecucion ha sido de {}'.format(t_minmax))


El modelo KNN con escalado MinMax ha obtenido una media de resultado de 0.57
Como errores:
	RMSE	= 433.1623694099446
	MAE	= 318.05921888959796
El tiempo de ejecucion ha sido de 0.06151771545410156


# Estandarizacion

In [125]:
# Escalamos con estandarización
scaler_standard = StandardScaler()
x_train_standard = scaler_standard.fit_transform(x_train)
x_test_standard = scaler_standard.transform(x_test)

# Entrenamos de la misma manera que sin ajuste de hiperparámetros
t_inicio = time.time() # Guardamos el tiempo inicial
knn = KNeighborsRegressor()
#np.random.seed(42)  # Para la toma de decisión entre varios puntos
knn.fit(x_train_standard, y_train)   # Se entrena el modelo
accuracy_train_standard = knn.score(x_test_standard, y_test)  # Se obtiene el resultado de aciertos
t_fin = time.time() # Guardamos el tiempo final
print('El modelo KNN con estandarización por omisión ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_standard))
# Miramos sus errores
prediction = knn.predict(x_test_standard)
error_rmse_standard = sqrt(mean_squared_error(y_test, prediction))
error_mae_standard = mean_absolute_error(y_test, prediction)

print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_standard, error_mae_standard))
t_standard = t_fin - t_inicio
print('El tiempo de ejecucion ha sido de {}'.format(t_standard))



El modelo KNN con estandarización por omisión ha obtenido una media de resultado de 0.62
Como errores:
	RMSE	= 411.92377576446233
	MAE	= 298.94642501595405
El tiempo de ejecucion ha sido de 0.012518167495727539


# Robusto

In [126]:
# Se hace un escalado robusto
scaler_robust = RobustScaler()
x_train_robust = scaler_robust.fit_transform(x_train)
x_test_robust = scaler_robust.transform(x_test)

# Entrenamos de la misma manera que sin ajuste de hiperparámetros
t_inicio = time.time() # Guardamos el tiempo inicial
knn = KNeighborsRegressor()
#np.random.seed(42)  # Para la toma de decisión entre varios puntos
knn.fit(x_train_robust, y_train)   # Se entrena el modelo
accuracy_train_robust = knn.score(x_test_robust, y_test)  # Se obtiene el resultado de aciertos
t_fin = time.time() # Guardamos el tiempo final
print('El modelo KNN con escalado robusto ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_robust))
# Miramos sus errores
prediction = knn.predict(x_test_robust)
error_rmse_robust = sqrt(mean_squared_error(y_test, prediction))
error_mae_robust = mean_absolute_error(y_test, prediction)
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_robust, error_mae_robust))
t_robust = t_fin - t_inicio
print('El tiempo de ejecucion ha sido de {}'.format(t_robust))

El modelo KNN con escalado robusto ha obtenido una media de resultado de 0.59
Como errores:
	RMSE	= 426.84002794164616
	MAE	= 310.4702271857052
El tiempo de ejecucion ha sido de 0.014000654220581055


Imprimimos todos los resultados:

In [127]:
print('El modelo KNN con hiperparámetros por omisión ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_og))
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_og, error_mae_og))
print('El tiempo de ejecucion ha sido de {}'.format(t_og))
print("")
print('El modelo KNN con escalado MinMax ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_minmax))
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_minmax, error_mae_minmax))
print('El tiempo de ejecucion ha sido de {}'.format(t_minmax))
print("")
print('El modelo KNN con estandarización por omisión ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_standard))
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_standard, error_mae_standard))
print('El tiempo de ejecucion ha sido de {}'.format(t_standard))
print("")
print('El modelo KNN con escalado robusto ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_robust))
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_robust, error_mae_robust))
print('El tiempo de ejecucion ha sido de {}'.format(t_robust))

El modelo KNN con hiperparámetros por omisión ha obtenido una media de resultado de 0.62
Como errores:
	RMSE	= 411.92377576446233
	MAE	= 298.94642501595405
El tiempo de ejecucion ha sido de 0.0

El modelo KNN con escalado MinMax ha obtenido una media de resultado de 0.57
Como errores:
	RMSE	= 433.1623694099446
	MAE	= 318.05921888959796
El tiempo de ejecucion ha sido de 0.06151771545410156

El modelo KNN con estandarización por omisión ha obtenido una media de resultado de 0.62
Como errores:
	RMSE	= 411.92377576446233
	MAE	= 298.94642501595405
El tiempo de ejecucion ha sido de 0.012518167495727539

El modelo KNN con escalado robusto ha obtenido una media de resultado de 0.59
Como errores:
	RMSE	= 426.84002794164616
	MAE	= 310.4702271857052
El tiempo de ejecucion ha sido de 0.014000654220581055


EVALUAR SI NECESITAMOS ESTUDIAR:
- DISTANCIA DE MAHALANOBIS
- ATRIBUTOS IRRELEVANTE
- DISTANCIA DE MINKOWSKY

# Segunda forma de eleccion de Escalado
# Número de vecino

In [128]:
# Definir el pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor())
])

# Definir el grid de parámetros
param_grid = {
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()],
    'knn__n_neighbors': [3, 5, 6, 9]
}

# Definir la división de series temporales
tscv = TimeSeriesSplit(n_splits=5)

# Realizar la búsqueda en la grilla con la división de series temporales
grid_search = GridSearchCV(pipe, param_grid, cv=tscv, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)

# Imprimir los mejores parámetros
print("Mejores parámetros: ", grid_search.best_params_)

# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# Realizar predicciones
y_pred = best_model.predict(x_test)

# Calcular e imprimir la métrica de rendimiento (RMSE)
print("RMSE: ", sqrt(mean_squared_error(y_test, y_pred)))

Mejores parámetros:  {'knn__n_neighbors': 9, 'scaler': StandardScaler()}
RMSE:  406.44896186523437


# Segunda Forma de número de vecino con el Escalado seleccionado (MIRAR)

In [129]:
# Normalizar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

# Definir el modelo KNN
knn = KNeighborsRegressor()

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_neighbors': range(1, 21)
}

# Inicializar GridSearchCV con TimeSeriesSplit
cv = TimeSeriesSplit(n_splits=5)
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error')

# Entrenar el modelo utilizando GridSearchCV
t_inicio = time.time() 
grid_search.fit(X_train_scaled, y_train)
t_fin = time.time()

# Obtener el mejor número de vecinos encontrado
best_n_neighbors = grid_search.best_params_['n_neighbors']
print("El número óptimo de vecinos encontrado es:", best_n_neighbors)

# Entrenar el modelo KNN con el número óptimo de vecinos
knn_best = KNeighborsRegressor(n_neighbors=best_n_neighbors)
knn_best.fit(X_train_scaled, y_train)

# Evaluar el modelo en el conjunto de prueba
accuracy_best = grid_search.score(X_test_scaled, y_test)
print('El modelo KNN con {} vecinos ha obtenido una media de resultado de {:.2f}'.format(best_n_neighbors, accuracy_best))

# Calcular errores
prediction_best = knn_best.predict(X_test_scaled)
error_rmse_best = sqrt(mean_squared_error(y_test, prediction_best))
error_mae_best = mean_absolute_error(y_test, prediction_best)
print('Como errores:\n\tRMSE\t= {:.2f}\n\tMAE\t= {:.2f}'.format(error_rmse_best, error_mae_best))

# Calcular el tiempo de ejecución
t_best = t_fin - t_inicio
print('El tiempo de ejecución ha sido de {:.2f} segundos'.format(t_best))

El número óptimo de vecinos encontrado es: 9
El modelo KNN con 9 vecinos ha obtenido una media de resultado de -165200.76
Como errores:
	RMSE	= 406.45
	MAE	= 301.03
El tiempo de ejecución ha sido de 1.40 segundos


# Conclusion
Pudiendo mejorar mucho más la predicción en 5 puntos.

In [130]:
# Normalizar los datos
scaler = StandardScaler()
x_train_final = scaler.fit_transform(x_train)
x_test_final = scaler.transform(x_test)


# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_neighbors': [9]
}

# Creamos el modelo
t_inicio = time.time() # Guardamos el tiempo inicial
knn = KNeighborsRegressor()

cv = TimeSeriesSplit(n_splits=5)

grid = GridSearchCV(knn, param_grid, cv=cv)
grid.fit(x_train_final, y_train)

# Evaluar el modelo en el conjunto de prueba
accuracy_train_final = grid.score(x_test_final, y_test)
print('El modelo KNN  ha obtenido una media de resultado de {:.2f}'.format(accuracy_best))

# Calcular errores
prediction_best = knn_best.predict(x_test_final)
error_rmse_final = sqrt(mean_squared_error(y_test, prediction_best))
error_mae_final = mean_absolute_error(y_test, prediction_best)
print('Como errores:\n\tRMSE\t= {:.2f}\n\tMAE\t= {:.2f}'.format(error_rmse_best, error_mae_best))

# Calcular el tiempo de ejecución
t_final = t_fin - t_inicio
print('El tiempo de ejecución ha sido de {:.2f} segundos'.format(t_best))



El modelo KNN  ha obtenido una media de resultado de -165200.76
Como errores:
	RMSE	= 406.45
	MAE	= 301.03
El tiempo de ejecución ha sido de 1.40 segundos


In [131]:
print('El modelo KNN con hiperparámetros por omisión ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_og))
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_og, error_mae_og))
print('El tiempo de ejecucion ha sido de {}'.format(t_og))
print("")
print('El modelo KNN ha obtenido una media de resultado de {:.2f}'.format(accuracy_train_final))
print('Como errores:\n\tRMSE\t= {}\n\tMAE\t= {}'.format(error_rmse_final, error_mae_final))
print('El tiempo de ejecucion ha sido de {}'.format(t_final))

El modelo KNN con hiperparámetros por omisión ha obtenido una media de resultado de 0.62
Como errores:
	RMSE	= 411.92377576446233
	MAE	= 298.94642501595405
El tiempo de ejecucion ha sido de 0.0

El modelo KNN ha obtenido una media de resultado de 0.63
Como errores:
	RMSE	= 406.44896186523437
	MAE	= 301.02940792739133
El tiempo de ejecucion ha sido de -0.051026105880737305


FIN KNN